## Part 3:

In [2]:
import time
from tqdm import tqdm
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer, ChiSqSelector, Normalizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.classification import LinearSVC, OneVsRest
from pyspark.ml import Pipeline


In [3]:
# Set Up Spark Session
spark = SparkSession.builder.appName("TextClassificationWithProgress").getOrCreate()

# Load Data
textDF = spark.read.json("hdfs:///user/dic24_shared/amazon-reviews/full/reviews_devset.json")
textDF.createOrReplaceTempView("review")
reviewTextDF = spark.sql("SELECT category, reviewText FROM review")

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
24/05/27 09:43:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/27 09:43:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/27 09:43:07 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [4]:
# Preprocessing Pipeline Functions
# Pipeline Step 1: Preprocessing and Tokenization
regexTokenizer = RegexTokenizer().setInputCol("reviewText").setOutputCol("terms").setPattern("[\\s_\\*+-:,;+=/\\\\%€[0-9]$&§@#~`\"'\\.()\\t\\[\\]{}!?]")

# Pipeline Step 2: Stopword Removal
remover = StopWordsRemover().setInputCol("terms").setOutputCol("filteredTerms")

# Pipeline Step 3: Count Vectorization
countVec = CountVectorizer().setInputCol("filteredTerms").setOutputCol("features").setVocabSize(20000).setMinDF(5)

# Pipeline Step 4: TF-IDF Calculation
idf = IDF().setInputCol("features").setOutputCol("weightedFeatures")

# Pipeline Step 5: Category Encoding
encoder = StringIndexer().setInputCol("category").setOutputCol("categoryIndex")

# Pipeline Step 6: Chi-Square Selection
selector = ChiSqSelector().setNumTopFeatures(2000).setFeaturesCol("weightedFeatures").setLabelCol("categoryIndex").setOutputCol("selectedFeatures")

# Pipeline Step 7: Normalization
normalizer = Normalizer().setInputCol("selectedFeatures").setOutputCol("normFeatures")

# Pipeline Step 8: SVM Classifier
svm = LinearSVC().setMaxIter(10).setRegParam(0.1).setFeaturesCol("normFeatures").setLabelCol("categoryIndex")

# Pipeline Step 9: OneVsRest for Multi-Class Classification
ovr = OneVsRest(classifier=svm, labelCol="categoryIndex", featuresCol="normFeatures")

In [5]:
# Defining the Pipeline here
pipeline = Pipeline(stages=[
        regexTokenizer,
        remover,
        countVec,
        idf,
        encoder,
        selector,
        normalizer,
        ovr
    ])

In [6]:
# model = pipeline.fit(reviewTextDF)
# result = pipeline.fit(reviewTextDF).transform(reviewTextDF)

In [7]:
downsampledDF = reviewTextDF.sample(withReplacement=False, fraction=0.1, seed=1234)

# Check the size of the downsampled dataset
print(f"Original count: {textDF.count()}")
print(f"Downsampled count: {downsampledDF.count()}")

Original count: 78829
Downsampled count: 8063


In [8]:
# Split the data into training and test sets
train_data, test_data = downsampledDF.randomSplit([0.8, 0.2], seed=1234)

In [9]:
# Grid Search for Hyperparameter Tuning
paramGrid = ParamGridBuilder() \
    .addGrid(svm.maxIter, [10, 50, 100]) \
    .addGrid(svm.regParam, [0.1, 0.01, 0.001]) \
    .build()

In [10]:
# Step 12: Cross-Validation
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(labelCol="categoryIndex", metricName="f1"),
                          numFolds=3)

In [11]:
start_time = time.time()

# Train the model using Cross-Validation
cv_model = crossval.fit(train_data)

# print timer
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Total time taken: {elapsed_time:.2f} seconds")

24/05/27 09:44:00 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/05/27 09:44:00 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
24/05/27 09:44:00 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/05/27 09:44:00 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


Total time taken: 4667.32 seconds


In [ ]:
# Make predictions on the test set
predictions = cv_model.transform(test_data)

In [13]:
# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="categoryIndex", predictionCol="prediction", metricName="f1")
f1_score = evaluator.evaluate(predictions)
print(f"F1 Score: {f1_score}")

F1 Score: 0.5566615387624674


In [15]:
predictions.select("category", "categoryIndex", "prediction").show(30, truncate=False)

+----------------+-------------+----------+
|category        |categoryIndex|prediction|
+----------------+-------------+----------+
|Apps_for_Android|11.0         |11.0      |
|Apps_for_Android|11.0         |0.0       |
|Apps_for_Android|11.0         |11.0      |
|Apps_for_Android|11.0         |11.0      |
|Apps_for_Android|11.0         |10.0      |
|Apps_for_Android|11.0         |11.0      |
|Apps_for_Android|11.0         |11.0      |
|Apps_for_Android|11.0         |11.0      |
|Apps_for_Android|11.0         |11.0      |
|Apps_for_Android|11.0         |11.0      |
|Apps_for_Android|11.0         |13.0      |
|Apps_for_Android|11.0         |11.0      |
|Apps_for_Android|11.0         |11.0      |
|Apps_for_Android|11.0         |11.0      |
|Apps_for_Android|11.0         |11.0      |
|Apps_for_Android|11.0         |11.0      |
|Apps_for_Android|11.0         |12.0      |
|Apps_for_Android|11.0         |11.0      |
|Apps_for_Android|11.0         |11.0      |
|Apps_for_Android|11.0         |

In [17]:
f1_scores = cv_model.avgMetrics

# Get the parameter grid
param_grid = crossval.getEstimatorParamMaps()

# Print the F1 scores along with their corresponding hyperparameters
print("F1 Scores for each parameter combination:")
for idx, (params, score) in enumerate(zip(param_grid, f1_scores)):
    print(f"Model {idx+1}: F1 Score = {score:.4f}, Params = {params}")

F1 Scores for each parameter combination:
Model 1: F1 Score = 0.5516, Params = {Param(parent='LinearSVC_059b37ad73a2', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LinearSVC_059b37ad73a2', name='regParam', doc='regularization parameter (>= 0).'): 0.1}
Model 2: F1 Score = 0.5434, Params = {Param(parent='LinearSVC_059b37ad73a2', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LinearSVC_059b37ad73a2', name='regParam', doc='regularization parameter (>= 0).'): 0.01}
Model 3: F1 Score = 0.5429, Params = {Param(parent='LinearSVC_059b37ad73a2', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LinearSVC_059b37ad73a2', name='regParam', doc='regularization parameter (>= 0).'): 0.001}
Model 4: F1 Score = 0.5438, Params = {Param(parent='LinearSVC_059b37ad73a2', name='maxIter', doc='max number of iterations (>= 0).'): 50, Param(parent='LinearSVC_059b37ad73a2', name='regParam', doc='regularization parameter (>= 0).'): 